In [ ]:
import cv2

def close_window_os():
    cv2.destroyAllWindows()
    for i in range (1,5): #trick for closing the display window on macos
        cv2.waitKey(1)

In [ ]:
# Problem 1

import os
import cv2
import pickle
import numpy as np

def getFaceLocation():

    camera = cv2.VideoCapture(0)
    facecascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    ret = True

#     #Refresh the camera
    for i in range(5):
        ret, frame = camera.read()

    while(ret):
        ret, frame = camera.read()
        if ret == True:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            face_coordinates = facecascade.detectMultiScale(gray, 1.3, 4)


            cv2.imshow('frames', frame)
            cv2.waitKey(1)
            if len(face_coordinates) > 0:
                (a, b, w, h) = face_coordinates[0]
                camera.release()
                close_window_os()

                return frame[b:b+h, a:a+w,:], (a, b, w, h)
        else:
            print('error')
            break


face_image, face_location = getFaceLocation()


In [ ]:
#Problem 2
def face_tracking(template, method):
    ret = True
    camera = cv2.VideoCapture(0)

    #Refresh the camera
    for i in range(5):
        ret, frame = camera.read()


    # Perform match operations.
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    while(ret):
        ret, frame = camera.read()
        if ret == True:
            img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Store width and height of template in w and h


        w, h = template.shape[::-1]
        res = cv2.matchTemplate(img_gray, template, method)

        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
        if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
            top_left = min_loc
        else:
            top_left = max_loc

        bottom_right = (top_left[0] + w, top_left[1] + h)

        cv2.rectangle(frame,top_left, bottom_right, 255, 2)
        cv2.imshow('template', template)
        cv2.imshow('frames', frame)

        #update template ......
#         template = frame[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0],:]

        if cv2.waitKey(1) == 27:
            break
    camera.release()
    close_window_os()

In [ ]:
face_tracking(face_image, cv2.TM_CCOEFF_NORMED)

In [ ]:
close_window_os()

In [ ]:
#problem 3

import numpy as np
import cv2 as cv
import argparse

cap = cv.VideoCapture(0)
# cap = cv.VideoCapture("/Users/vutl2004gmail.com/Downloads/slow_traffic_small.mp4")

#Refresh the camera
for i in range(5):
    ret, frame = cap.read()

# take first frame of the video
ret,frame = cap.read()

# setup initial location of window
x, y, w, h = face_location # simply hardcoded the values
track_window = (x, y, w, h)

# set up the ROI for tracking
roi = frame[y:y+h, x:x+w]


hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret, frame = cap.read()

    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv.meanShift(dst, track_window, term_crit)

        # Draw it on image
        x,y,w,h = track_window
        img2 = cv.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv.imshow('img2',img2)

        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break

cap.release()
close_window_os()


In [ ]:
import numpy as np
import cv2 as cv
import argparse

def object_tracking_camera_file():
    cap = cv.VideoCapture(0)

    #Refresh the camera
    for i in range(5):
        ret, frame = cap.read()

    # take first frame of the video
    ret,frame = cap.read()


    # setup initial location of window
    x, y, w, h = (524, 309, 293, 293) # simply hardcoded the values
    track_window = (x, y, w, h)

    # set up the ROI for tracking
    roi = frame[y:y+h, x:x+w]

    hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
    mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
    roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
    cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)

    # Setup the termination criteria, either 10 iteration or move by at least 1 pt
    term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )

    while(1):
        ret, frame = cap.read()

        if ret == True:
            hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
            dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)

            # apply meanshift to get the new location
            ret, track_window = cv.meanShift(dst, track_window, term_crit)
#             ret, track_window = cv.CamShift(dst, track_window, term_crit)


            # Draw it on image
            x,y,w,h = track_window
            img2 = cv.rectangle(frame, (x,y), (x+w,y+h), (255, 255, 0) ,4)
            cv.imshow('img2',img2)

            k = cv.waitKey(30) & 0xff
            if k == 27:
                break
        else:
            break

    cap.release()
    close_window_os()

object_tracking_camera_file()

In [ ]:
import numpy as np
import cv2 as cv
import argparse

def object_tracking_video_file():
    cap = cv.VideoCapture("/Users/vutl2004gmail.com/Downloads/8169005713089002738.mp4")

    # take first frame of the video
    ret,frame = cap.read()


    # setup initial location of window
    x, y, w, h = (300, 400, 200, 200) # simply hardcoded the values
    track_window = (x, y, w, h)

    # set up the ROI for tracking
    roi = frame[y:y+h, x:x+w]

    hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
    mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
    roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
    cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)

    # Setup the termination criteria, either 10 iteration or move by at least 1 pt
    term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )

    history = []

    ret, frame = cap.read()
    height, width, layers = frame.shape

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 file
    fps = 30  # Frames per second
    video_file = 'output_video.mp4'
    out = cv2.VideoWriter(video_file, fourcc, fps, (width, height))



    while(1):
        ret, frame = cap.read()

        if ret == True:
            hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
            dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)

            # apply meanshift to get the new location
            ret, track_window = cv.meanShift(dst, track_window, term_crit)
#             ret, track_window = cv.CamShift(dst, track_window, term_crit)



            # Draw it on image
            x,y,w,h = track_window
            img2 = cv.rectangle(frame, (x,y), (x+w,y+h), (255, 255, 0) ,4)
            history.append((x+int(w/2),y+int(h/2)))

            if (len(history) >= 2):
                for i in range(len(history)-1):
                    (x,y) = history[i]
                    img2 = cv2.line(frame, history[i], history[i+1], (0,0,255), 2)


            cv.imshow('img2',img2)
            out.write(img2)

            k = cv.waitKey(1)
#             k = cv.waitKey(0) & 0xff
#             if k == 27:
#                 break
        else:
            break
    out.release()
    print(len(history))
    cap.release()
    close_window_os()

object_tracking_video_file()

875
